# Step 0 - Loading the necessary libraries and setting them up

In [35]:
%matplotlib inline

import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sn
import matplotlib
matplotlib.use('Qt5Agg')

import matplotlib.pyplot as plt

import mpld3
mpld3.enable_notebook()

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', 2)
pd.set_option('display.max_columns', 7)

# Step 1 - Data Collection

In [36]:
A = pd.read_csv('A.csv', parse_dates=True, index_col=0)
A

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2015-05-15,18251.970703,18272.720703,18215.070313,18272.560547,18272.560547,108220000
...,...,...,...,...,...,...
2020-05-14,23049.060547,23630.859375,22789.619141,23625.339844,23625.339844,472700000


In [37]:
B = pd.read_csv('B.csv', parse_dates=True, index_col=0); 
B

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2015-05-15,2122.070068,2123.889893,2116.810059,2122.72998,2122.72998,3092080000
...,...,...,...,...,...,...
2020-05-14,2794.540039,2852.800049,2766.639893,2852.50000,2852.50000,5641920000


In [38]:
C = pd.read_csv('C.csv', parse_dates=True, index_col=0); 
C

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2015-05-15,12.46,13.090000,12.350000,12.380000,12.380000,0
...,...,...,...,...,...,...
2020-05-14,35.16,39.279999,32.330002,32.610001,32.610001,0


# Step 2 - Data wrangling/munging

In [39]:
merged_df = A.join(B, how='outer', lsuffix='_A', sort=True).join(C, how='outer', lsuffix='_B', rsuffix='_C', sort=True)

In [40]:
merged_df

,Open_A,High_A,Low_A,...,Close_C,Adj Close_C,Volume_C
Date,,,,,,,
2015-05-15,18251.970703,18272.720703,18215.070313,...,12.380000,12.380000,0.0
...,...,...,...,...,...,...,...
2020-05-14,23049.060547,23630.859375,22789.619141,...,32.610001,32.610001,0.0


# Step 3 - Data cleaning

In [41]:
merged_df[merged_df.isnull().all(axis=1)]

,Open_A,High_A,Low_A,...,Close_C,Adj Close_C,Volume_C
Date,,,,,,,


In [42]:
merged_df[['Close_A', 'Close_B', 'Close_C']].isnull().any(axis=1).sum()

148

In [43]:
valid_close_df = merged_df.dropna(subset=['Close_A', 'Close_B', 'Close_C'], how='any')

In [44]:
valid_close_df[['Close_A', 'Close_B', 'Close_C']].isnull().any(axis=1).sum()

0

In [45]:
valid_close_df

,Open_A,High_A,Low_A,...,Close_C,Adj Close_C,Volume_C
Date,,,,,,,
2015-05-15,18251.970703,18272.720703,18215.070313,...,12.380000,12.380000,0.0
...,...,...,...,...,...,...,...
2020-05-14,23049.060547,23630.859375,22789.619141,...,32.610001,32.610001,0.0


In [46]:
valid_close_df.isnull().any(axis=1).sum()

165

In [47]:
valid_close_df[valid_close_df.isnull().any(axis=1)]

,Open_A,High_A,Low_A,...,Close_C,Adj Close_C,Volume_C
Date,,,,,,,
2015-05-18,18267.250000,18325.539063,18244.259766,...,12.730000,12.730000,0.0
...,...,...,...,...,...,...,...
2020-05-01,24120.779297,24120.779297,23645.300781,...,37.189999,37.189999,0.0


In [48]:
valid_close_complete = valid_close_df.fillna(method='backfill')

In [49]:
valid_close_complete.isnull().any(axis=1).sum()

0

# Step 4 - Obtaining Descriptive Statistics

In [50]:
pd.set_option('display.max_rows', None)
valid_close_complete.describe()

,Open_A,High_A,Low_A,...,Close_C,Adj Close_C,Volume_C
count,1111.000000,1111.000000,1111.000000,...,1111.000000,1111.000000,1111.0
mean,22291.125036,22402.395046,22168.922744,...,16.417606,16.417606,0.0
std,3771.056417,3784.558787,3755.740756,...,8.016538,8.016538,0.0
min,15676.259766,15897.820313,15370.330078,...,9.140000,9.140000,0.0
25%,18232.280274,18285.384766,18156.614258,...,12.300000,12.300000,0.0
50%,22762.029297,22872.890625,22634.449219,...,14.090000,14.090000,0.0
75%,25516.320312,25659.810547,25382.705078,...,17.665000,17.665000,0.0
max,29440.470703,29568.570313,29406.750000,...,82.690002,82.690002,0.0


In [51]:
pd.set_option('display.max_rows', 2)

In [52]:
prices_only = valid_close_complete.drop(['Volume_A', 'Volume_B', 'Volume_C'], axis=1)

In [53]:
prices_only

,Open_A,High_A,Low_A,...,Low_C,Close_C,Adj Close_C
Date,,,,,,,
2015-05-15,18251.970703,18272.720703,18215.070313,...,12.350000,12.380000,12.380000
...,...,...,...,...,...,...,...
2020-05-14,23049.060547,23630.859375,22789.619141,...,32.330002,32.610001,32.610001


# Step 5 - Visual Inspection of the Data

In [67]:
valid_close_complete['Open_A'].plot(figsize=(12,6), linestyle='--', color='black', legend='Open_A')
valid_close_complete['Close_A'].plot(figsize=(12,6), linestyle='-', color='grey', legend='Close_A')
valid_close_complete['Low_A'].plot(figsize=(12,6), linestyle=':', color='black', legend='Low_A')
valid_close_complete['High_A'].plot(figsize=(12,6), linestyle='-.', color='grey', legend='High_A')

<Axes: xlabel='Date'>

In [68]:
valid_close_complete['Open_C'].plot(figsize=(12,6), linestyle='--', color='black', legend='Open_C')
valid_close_complete['Close_C'].plot(figsize=(12,6), linestyle='-', color='grey', legend='Close_C')
valid_close_complete['Low_C'].plot(figsize=(12,6), linestyle=':', color='black', legend='Low_C')
valid_close_complete['High_C'].plot(figsize=(12,6), linestyle='-.', color='grey', legend='High_C')

<Axes: xlabel='Date'>

# Step 6 - Data Cleaning

In [69]:
no_outlier_prices = prices_only[(np.abs(stats.zscore(prices_only)) < 6).all(axis=1)]

In [70]:
no_outlier_prices['Open_C'].plot(figsize=(12,6), linestyle='--', color='black', legend='Open_C')
no_outlier_prices['Close_C'].plot(figsize=(12,6), linestyle='-', color='grey', legend='Close_C')
no_outlier_prices['Low_C'].plot(figsize=(12,6), linestyle=':', color='black', legend='Low_C')
no_outlier_prices['High_C'].plot(figsize=(12,6), linestyle='-.', color='grey', legend='High_C')

<Axes: xlabel='Date'>

In [71]:
pd.set_option('display.max_rows', None)
no_outlier_prices[['Open_C', 'Close_C', 'Low_C', 'High_C']].describe()

,Open_C,Close_C,Low_C,High_C
count,1095.000000,1095.000000,1095.000000,1095.000000
mean,16.147571,16.072648,15.223635,17.214539
std,6.764147,6.773569,5.995822,7.588690
min,9.010000,9.140000,8.560000,9.310000
25%,12.420000,12.285000,11.865000,12.890000
50%,14.160000,14.060000,13.440000,14.940000
75%,17.625000,17.500000,16.550000,18.929999
max,65.669998,63.950001,58.029999,68.860001


In [59]:
pd.set_option('display.max_rows', 5)

# Step 7 - Advanced Visualization Techniques

In [72]:
close_prices = no_outlier_prices[['Close_A', 'Close_B', 'Close_C']]

In [73]:
delta_close_prices = (close_prices.shift(-1) - close_prices).fillna(0)
delta_close_prices.columns = ['Delta_Close_A', 'Delta_Close_B', 'Delta_Close_C']

In [74]:
delta_close_prices

,Delta_Close_A,Delta_Close_B,Delta_Close_C
Date,,,
2015-05-15,26.320312,6.469971,0.350000
2015-05-18,13.509766,-1.369873,0.120000
2015-05-19,-26.650391,2.989990,-0.740000
2015-05-21,-53.720703,-4.760009,0.020000
2015-05-22,-190.480468,-21.860108,1.930000
2015-05-26,121.451171,19.280029,-0.790000
2015-05-27,-36.871093,-2.689941,0.040000
2015-05-28,-115.439453,-13.400146,0.530000
2015-05-29,29.689453,4.340087,0.130000


In [75]:
pd.set_option('display.max_rows', None)

In [76]:
delta_close_prices.describe()

,Delta_Close_A,Delta_Close_B,Delta_Close_C
count,1095.000000,1095.000000,1095.000000
mean,4.888383,0.666457,0.018475
std,268.137091,29.218995,1.938761
min,-2848.310547,-294.049805,-9.120001
25%,-70.894532,-7.140076,-0.760000
50%,15.539063,1.770020,-0.080000
75%,113.290039,12.255005,0.540001
max,1351.619141,154.510009,20.010001


In [77]:
delta_close_prices['Delta_Close_A'].plot(kind='hist', bins=100, figsize=(12,6), color='black', grid=True)

<Axes: ylabel='Frequency'>

In [78]:
delta_close_prices['Delta_Close_B'].plot(kind='box', figsize=(12,6), color='black', grid=True)

<Axes: ylabel='Frequency'>

In [79]:
pd.plotting.scatter_matrix(delta_close_prices, figsize=(10,10), color='black', alpha=0.75, diagonal='kde', grid=True)

array([[<Axes: xlabel='Delta_Close_A', ylabel='Delta_Close_A'>,
        <Axes: xlabel='Delta_Close_B', ylabel='Delta_Close_A'>,
        <Axes: xlabel='Delta_Close_C', ylabel='Delta_Close_A'>],
       [<Axes: xlabel='Delta_Close_A', ylabel='Delta_Close_B'>,
        <Axes: xlabel='Delta_Close_B', ylabel='Delta_Close_B'>,
        <Axes: xlabel='Delta_Close_C', ylabel='Delta_Close_B'>],
       [<Axes: xlabel='Delta_Close_A', ylabel='Delta_Close_C'>,
        <Axes: xlabel='Delta_Close_B', ylabel='Delta_Close_C'>,
        <Axes: xlabel='Delta_Close_C', ylabel='Delta_Close_C'>]],
      dtype=object)

In [ ]:
delta_close_prices.corr()

In [ ]:
plt.figure(figsize=(6,6))
sn.heatmap(delta_close_prices.corr(), annot=True, square=True, linewidths=2)

# Special Python Libraries for EDA

In [80]:
import dtale
dtale.show(valid_close_df, ignore_duplicate=True)

ModuleNotFoundError: No module named 'dtale'